In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import  tqdm_notebook

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls data


men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [15]:
df['brand_cat']= df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [16]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [19]:
df.features.head()

0    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2    [{"key":"Gender","value":["Men"]},{"key":"Colo...
3    [{"key":"Gender","value":["Men"]},{"key":"Colo...
4    [{"key":"Gender","value":["Men"]},{"key":"Colo...
Name: features, dtype: object

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [33]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [38]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [39]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_carats', 'feat_used with equipment type', 'feat_genre',
       'feat_isbn', 'feat_waist size', 'feat_authenticity', 'feat_shade',
       'feat_sports league', 'feat_quantity in set', 'feat_shipping to usa'],
      dtype='object', length=526)

In [48]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [82]:
 df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
# def 

# def convert_to_grams(weight):
#   if 'g' in weight
#     return weight
#   elif 

# for key in tqdm_notebook(keys):
#   df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df['feat_brand_cat']= df['feat_brand'].factorize()[0]
df['feat_color_cat']= df['feat_color'].factorize()[0]
df['feat_gender_cat']= df['feat_gender'].factorize()[0]
df['feat_manufacturer part number']= df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']= df['feat_material'].factorize()[0]

df['feat_sport_cat']= df['feat_sport'].factorize()[0]
df['feat_style_cat']= df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [51]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape[0]

8846

In [0]:
feats = ['']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'],model)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [0]:
feats = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_sport_cat', 'feat_style_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [81]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.315865295886454, 4.215190109198238)


Weight,Feature
0.2556 ± 0.0080,brand_cat
0.1033 ± 0.0075,feat_material_cat
0.0255 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0093 ± 0.0015,feat_metal type_cat
0.0030 ± 0.0005,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [65]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'brand': 'nike'},
       {'sport': 'skate shoes', 'occasion': 'skate shoes', 'material': 'nubuck', 'gender': 'men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'nubuck', 'color': 'unvrsty rd/mdnght trq wht gm l', 'model': '333824 613', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'skateboarding shoes', 'manufacturer part number': '333824 613', 'brand': 'nike', 'shoe width': 'd(m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'dark obsidian/black/mid navy/black', 'casual & dress shoe style': 'running shoes', 'shoe width': 'medium (d, m)'},
       {'style': 'athletic sneakers', 'condition': 'new with box'},
       {'sport': 'soccer', 'condition': 'new with box'}], dtype=object)

In [0]:
!git add matrix_one/day5.ipynb

In [84]:
!git commit -m "added new features to model"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@58eb0ab4686d.(none)')


In [0]:
!git config --global user.email "kchowicki1@gmail.com"
!git config --global user.name "Kacper"


In [87]:
!git push

Everything up-to-date
